# TODO

## Data Sections
- Exp-to-Non exp productivity ratio matrix
- Domestic Owned Exporter / Non-Exporter KS [All Ps]
- Exporter / Non-Exporter KS for all sizes [All Ps]
- All Firms Exp / nExp KS [Comparison of Ps]


## Hypotheses
- Exporter >= Non-Exporter [All categories]
- Smaller firms are more sensitive to export status


In [33]:
import pandas as pd
import numpy as np
from scipy.stats import kstest

import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
fmt_df = pd.DataFrame(
    index=[
        "p10",
        "p25",
        "p50",
        "p75",
        "p90",
    ]
)
df = pd.read_csv("data.csv", encoding="utf-8", low_memory=False)
df.columns = df.columns.str.strip()

In [3]:
df.loc[df['Dimension'] == '2. Sizeband']

,Year,Metric,Dimension,Category,Status,Source,Mean,SD,p10,p25,p50,p75,p90,N
60,2011,GVA per worker,2. Sizeband,Large (250+),Exporter,ABS,"96,500","307,500","18,500","37,000","64,500","107,000","176,500","2,150"
61,2012,GVA per worker,2. Sizeband,Large (250+),Exporter,ABS,"97,500","255,500","19,000","37,000","64,500","107,500","182,000","2,250"
62,2013,GVA per worker,2. Sizeband,Large (250+),Exporter,ABS,"96,000","216,500","20,000","39,000","67,000","109,500","178,500","2,400"
63,2014,GVA per worker,2. Sizeband,Large (250+),Exporter,ABS,"99,000","207,000","22,500","40,500","70,000","114,000","183,500","2,500"
64,2015,GVA per worker,2. Sizeband,Large (250+),Exporter,ABS,"97,500","217,000","24,000","41,000","68,000","110,500","184,500","2,600"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2018,GVA per worker,2. Sizeband,Small (10-49),Non-Exporter,ABS-TiG,"42,500","100,000","7,000","14,500","29,000","54,000","86,000","140,000"
296,2019,GVA per worker,2. Sizeband,Small (10-49),Non-Exporter,ABS-TiG,"43,000","120,500","6,000","14,000","30,500","55,000","89,500","139,300"
297,2020,GVA per worker,2. Sizeband,Small (10-49),Non-Exporter,ABS-TiG,"39,500","144,500","2,000","8,500","25,000","50,000","85,500","144,850"
298,2021,GVA per worker,2. Sizeband,Small (10-49),Non-Exporter,ABS-TiG,"44,500","302,000","3,000","11,000","28,000","57,500","97,500","140,700"


In [19]:
kstest(
    df.loc[df['Status'] == 'Exporter', 'Mean'],
    df.loc[df['Status'] == 'Non-Exporter', 'Mean']
)

KstestResult(statistic=np.float64(0.6377777777777778), pvalue=np.float64(2.3803919609674216e-172), statistic_location=' 56,000 ', statistic_sign=np.int8(-1))

In [32]:
Qs = ["p10", "p25", "p50", "p75", "p90"]
sizes = ['Small (10-49)', 'Medium (50-249)', 'Large (250+)']
status = ["Exporter", "Non-Exporter"]

exp_nexp_df = fmt_df.copy()
exp_nexp_df['Small (10-49)'] = np.nan
exp_nexp_df['Medium (50-249)'] = np.nan
exp_nexp_df['Large (250+)'] = np.nan

for i, q in enumerate(Qs):
    for size in sizes:
        samp1 = df.loc[(df["Status"] == "Exporter") & (df['Category'] == size), q]
        samp2 = df.loc[(df["Status"] == "Non-Exporter") & (df['Category'] == size), q]
        exp_nexp_df.loc[q, size] = kstest(samp1, samp2).pvalue
print("Exporter vs Non-Exporter KS test per sizeband [PVAL]")
exp_nexp_df.round(6)

Exporter vs Non-Exporter KS test per sizeband [PVAL]


,Small (10-49),Medium (50-249),Large (250+)
p10,0.000000,0.000000,0.000006
p25,0.000000,0.000000,0.000001
p50,0.000000,0.000000,0.000001
p75,0.000293,0.000024,0.000900
p90,0.000000,0.000006,0.000900


In [ ]:
sizes = ['Small (10-49)', 'Medium (50-249)', 'Large (250+)']

sizecomp_exp_df = pd.DataFrame(
    columns=sizes,
    index=sizes
)

samp = df.loc[df['Status'] == 'Exporter']
for i in sizes:
    for j in sizes:
        samp1 = samp.loc[samp['Category'] == i, 'p50']
        samp2 = samp.loc[samp['Category'] == j, 'p50']
        sizecomp_exp_df.loc[i, j] = kstest(samp1, samp2).pvalue

print("Exporter Sizeband Comparison KS test [PVAL]")
sizecomp_exp_df.round(6)

Exporter Sizeband Comparison KS test [PVAL]


,Small (10-49),Medium (50-249),Large (250+)
Small (10-49),1.0,0.392945,0.000001
Medium (50-249),0.392945,1.0,0.000293
Large (250+),0.000001,0.000293,1.0


In [15]:
sizecomp_nexp_df = pd.DataFrame(
    columns=sizes,
    index=sizes
)

samp = df.loc[df['Status'] == 'Non-Exporter']
for i in sizes:
    for j in sizes:
        samp1 = samp.loc[samp['Category'] == i, 'p50']
        samp2 = samp.loc[samp['Category'] == j, 'p50']
        sizecomp_nexp_df.loc[i, j] = kstest(samp1, samp2).pvalue

print("Non-Exporter Sizeband Comparison KS test [PVAL]")
sizecomp_nexp_df.round(6)

Non-Exporter Sizeband Comparison KS test [PVAL]


,Small (10-49),Medium (50-249),Large (250+)
Small (10-49),1.0,0.000001,0.070888
Medium (50-249),0.000001,1.0,0.03458
Large (250+),0.070888,0.03458,1.0


In [28]:
Q_comp_df = pd.DataFrame(
    columns=['Foreign-Owned', 'Domestic-Owned'],
    index=Qs
)
for i in Qs:
    for j in Q_comp_df.columns:
        samp1 = df.loc[(df['Category'] == j) & (df['Status'] == 'Exporter'), i]
        samp2 = df.loc[(df['Category'] == j) & (df['Status'] == 'Non-Exporter'), i]
        Q_comp_df.loc[i, j] = kstest(samp1, samp2).pvalue

print("Exporter vs Non-Exporter KS test per ownership type [PVAL]")
Q_comp_df.round(6)

Exporter vs Non-Exporter KS test per ownership type [PVAL]


,Foreign-Owned,Domestic-Owned
p10,0.000006,0.000001
p25,0.0,0.0
p50,0.0,0.0
p75,0.594071,0.0
p90,0.594071,0.000001
